In [ ]:
# make PSTH

song_use = 0
# this_ind = song_mask==song_use+1
# this_t = song_ramp[this_ind]
# plt.hist(this_t)
PSTH = get_STA.get_PSTH(spk_times,song_use,song_mask,song_ramp)
#plt.hist(tmp)


# plt.plot(PSTH)
tmp = np.unique(song_ramp[song_mask==song_use+1])
np.max(tmp)

import get_STA
reload(get_STA)
#_,_,quick_spect = get_STA.reduced_spectrogram(songs[song_use])
window = 20
quick_spect = get_STA.flat_reduced_spectrogram(songs[song_use],window)
print(np.shape(PSTH),np.shape(quick_spect))
quick_spectT = np.transpose(quick_spect)


import theano
import mner.optimizer
import mner.util.util
import mner.solvers.solvers
import mner.solvers.constraints
import mner.solvers.samplers

s, s_avg, s_std = mner.util.util.zscore_features(quick_spectT)
y = PSTH[window-1:]

jack = 1
njack = 4

# fraction of the data reserved for training and cross-validation (remainder is testing)
train_fraction = 0.7
cv_fraction = 0.2
nsamp = np.shape(s)[0]
ndim = np.shape(s)[1]
print(np.shape(s),ndim)

trainset, cvset, testset, nshift = mner.util.util.generate_dataset_logical_indices(train_fraction, cv_fraction, nsamp, njack)
trainset, cvset, testset = mner.util.util.roll_dataset_logical_indices(trainset, cvset, testset, nshift, jack-1)
datasets = {'trainset': trainset, 'cvset': cvset, 'testset': testset}

# model parameters
rank = 6
cetype = ["UV-linear-insert"]
rtype = ["nuclear-norm"]

# if J is symmetrized using linear constraints, need to set signs of eigenvalues
csigns = np.array([1, -1]*(rank/2))

# set scaling of cost function (for each data set)
fscale = {"trainset": -1, "cvset": -1, "testset": -1}

# choose solver
#solver = mner.solvers.solvers.IPMSolver
solver = mner.solvers.solvers.LBFGSSolver

# fit parameters (note the change for demo_type == 7 below)
factr = 1.0e10
lbfgs = 30


  opt = mner.optimizer.Optimizer(y, s, rank, cetype=cetype, rtype=[], solver=solver, datasets=datasets, fscale=fscale, csigns=csigns, lbfgs=lbfgs, precompile=True, compute_hess=False, verbosity=2, iprint=1, factr=factr)


x, ftrain = opt.optimize()

a, h, U, V = mner.util.util.vec_to_weights(x, ndim, rank)
V = np.dot(U, np.diag(csigns))

# form J and symmetrize then compute components
Jsym = np.dot(U, V.T)
Jsym = 0.5*(Jsym + Jsym.T)
[u, _, _] = np.linalg.svd(Jsym)

nx = 20
ny = 16
cm = .2
plt.imshow(np.reshape(u[:,1], (ny, nx)), aspect='equal', interpolation='none',clim=(-cm, cm))
#plt.colorbar()